In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv
import random
from scipy.stats import norm
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.nn.functional import binary_cross_entropy
from sklearn.metrics import precision_recall_fscore_support
from torch.optim import lr_scheduler
from scipy import sparse
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import io, tarfile

datafile = "/vol/bitbucket/mjh24/IAEA-thesis/data/swde_HTMLgraphs.tar"

SEED = 16

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    

set_seed(SEED)

***BELOW***
If data-loading < 5-10 % of total epoch time with num_workers=0, stick with the simple path.
Otherwise, parallel loading with share-friendly torch_sparse.SparseTensor
almost always pays off.

In [12]:
# ───────────────────────────────────────────────────────── Tar-reader dataset
class TarGraphDataset(Dataset):
    """
    Each graph is stored under its own sub-directory *inside* one .tar:

        graphs.tar
        ├── 0001/X.npz
        ├── 0001/E.npz
        ├── 0001/edge_index.npy
        ├── 0001/labels.npz
        ├── 0001/label_index.npy
        ├── 0001/label_value.npy
        ├── 0002/…
        └── …

    The tar is opened once; __getitem__ streams the six files for graph *idx*
    straight into memory, converts them to native PyTorch tensors and returns.
    """

    def __init__(self, tar_path: str | Path):
        self.tar = tarfile.open(tar_path, mode="r:*")      # gzip/none/…
        self.index: dict[str, dict[str, tarfile.TarInfo]] = {}

        # Build a small lookup table in RAM  {gid: {filename: tarinfo}}
        for member in self.tar.getmembers():
            if not member.isfile():
                continue

            p     = Path(member.name)
            gid   = str(p.parent)   # '0007'
            fname = p.name          # 'X.npz'

            # keep only folders that really are 4-digit graph IDs
            if gid[-4:].isdigit():
                self.index.setdefault(gid, {})[fname] = member


        self.gids = sorted(self.index)

        # Remove thos with no labels
        for gid, files in self.index.items():
            if not files.get("labels.npz"):
                self.gids.remove(gid)

    # ------------- helpers --------------------------------------------------
    @staticmethod
    def _npz_to_csr(buf: bytes, dtype=torch.float32):
        csr = sparse.load_npz(io.BytesIO(buf)).tocsr()
        crow = torch.from_numpy(csr.indptr.astype(np.int64))
        col  = torch.from_numpy(csr.indices.astype(np.int64))
        val  = torch.from_numpy(csr.data).to(dtype)
        return torch.sparse_csr_tensor(
            crow, col, val, size=csr.shape, dtype=dtype, requires_grad=False
        )

    @staticmethod
    def _npy_to_tensor(buf: bytes, dtype):
        arr = np.load(io.BytesIO(buf), allow_pickle=False)
        return torch.from_numpy(arr).to(dtype)

    # ------------- Dataset API ---------------------------------------------
    def __len__(self):
        return len(self.gids)

    def __getitem__(self, idx):
        gid   = self.gids[idx]
        files = self.index[gid]

        get = lambda name: self.tar.extractfile(files[name]).read()
        
        
        fileinfo = gid

        X   = self._npz_to_csr(get("X.npz"),       dtype=torch.float32)
        Aef = self._npz_to_csr(get("E.npz"),       dtype=torch.float32)
        Lef = self._npz_to_csr(get("labels.npz"),  dtype=torch.float32)

        Aei = self._npy_to_tensor(get("edge_index.npy"),  dtype=torch.int64)
        Lei = self._npy_to_tensor(get("label_index.npy"), dtype=torch.int64)
        y   = self._npy_to_tensor(get("label_value.npy"), dtype=torch.int64)

        return fileinfo, X, Aei, Aef, Lei, Lef, y


# ───────────────────────────────────────────────────────── loader utilities
def identity_collate(batch):
    """batch == list of length 1 → return that single sample untouched."""
    return batch[0]

def make_loader(ds, batch_size=1, shuffle=False):
    assert batch_size == 1, "identity_collate works with batch_size=1"
    return DataLoader(ds,
                      batch_size=batch_size,
                      shuffle=shuffle,
                      collate_fn=identity_collate,
                      num_workers=0,      # ← stays serial
                      pin_memory=True)    # fast GPU transfer if needed


In [15]:
# dataset   = TarGraphDataset("/vol/bitbucket/mjh24/IAEA-thesis/data/swde_HTMLgraphs.tar")
# loader    = make_loader(dataset, batch_size=1, shuffle=False)

# count = 0
# for fileinfo, X, Aei, Aef, Lei, Lef, y in loader:
#     print(fileinfo)
#     count +=1
# print(count)

In [ ]:
# class LazyGraphDataset(Dataset):
#     """
#     Each graph lives in its own .npz / .pt / whatever on disk.
#     __getitem__ loads it just-in-time.
#     """

#     def __init__(self, folderpaths):
#         """
#         meta_csv: a CSV (or list of dicts) with columns:
#             path_X, path_A_index, path_A_feat, path_L_index, path_L_feat, path_y
#         Only these tiny strings stay in RAM.
#         """
#         self.folderpaths = list(folderpaths)

#     def _import_tensor(self, filename: str, dtype: torch.dtype, is_sparse: bool = False):
#         """
#         Load a .npz CSR matrix and return either
#         • a torch.sparse_csr_tensor              (if is_sparse=True)
#         • a torch.Tensor (dense)                 (otherwise)
#         """
#         csr = sparse.load_npz(filename).tocsr()

#         if is_sparse:
#             crow = torch.from_numpy(csr.indptr.astype(np.int64))
#             col  = torch.from_numpy(csr.indices.astype(np.int64))
#             val  = torch.from_numpy(csr.data).to(dtype)
#             return torch.sparse_csr_tensor(crow, col, val,size=csr.shape, dtype=dtype, requires_grad=False)
#         # — otherwise densify —
#         return torch.from_numpy(csr.toarray()).to(dtype)

#     def __getitem__(self, idx):
#         folder_path = self.folderpaths[idx]

#         X = self._import_tensor((folder_path/"X.npz"), torch.float32, is_sparse=False)
#         #A = self._import_tensor(folder_path/"A.npz", torch.long, True)
#         Aef = self._import_tensor((folder_path/"E.npz"), torch.float32, is_sparse=True)
#         Aei = torch.from_numpy(np.load((folder_path/"edge_index.npy")))
#         Lef = self._import_tensor((folder_path/"labels.npz"), torch.float32, is_sparse=True)
#         Lei = torch.from_numpy(np.load((folder_path/"label_index.npy")))
#         y = torch.from_numpy(np.load((folder_path/"label_value.npy")))

#         return X, Aei, Aef, Lei, Lef, y

#     def __len__(self):
#         return len(self.folderpaths)

# def graph_collate(batch):
#     # batch is a list of tuples
#     xs, aei, aef, lei, lef, ys = zip(*batch)   # tuples of length B

#     return (list(xs),                          # list of sparse X
#             list(aei),                         # list of edge_index
#             list(aef),                         # list of sparse A_edge_feat
#             list(lei),
#             list(lef),
#             list(ys))                          # dense y can still be list/stack

# def make_loader(dataset, batch_size=1, shuffle=False):
#     return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=graph_collate, num_workers=0, pin_memory=True)

In [ ]:
# def walk_limited(root: Path, max_depth: int, pat: str):
#     root_depth = len(root.parts)
#     for dirpath, dirnames, _ in os.walk(root):
#         depth = len(Path(dirpath).parts) - root_depth
#         if depth > max_depth:
#             # prune traversal
#             dirnames[:] = []
#             continue
#         for d in dirnames:
#             p = Path(dirpath, d)
#             if p.match(pat):
#                 yield p

# src = Path("/vol/bitbucket/mjh24/IAEA-thesis/data/swde_HTMLgraphs/movie/movie")
# batch_dirs = list(walk_limited(src, max_depth=2, pat='[0-9][0-9][0-9][0-9]'))
# print(src.exists())
# batchFiles = list(src.rglob("[0-9][0-9][0-9][0-9]"))
# print(len(batchFiles))
# dataset = LazyGraphDataset(batchFiles)
# dataloader = make_loader(dataset, batch_size=1, shuffle=False)

True
16000


In [ ]:
# for Xs, Aeis, Aefs, Leis, Lefs, ys in dataloader:
#     print(Xs[0].shape, Aeis[0].shape, Aefs[0].shape, Leis[0].shape, Lefs[0].shape, ys[0].shape)
#     break

In [ ]:
# Helper function to normalise the A matrix
def symmetric_normalize(A_tilde):
    """
    Performs symmetric normalization of A_tilde (Adj. matrix with self loops):
      A_norm = D^{-1/2} * A_tilde * D^{-1/2}
    Where D_{ii} = sum of row i in A_tilde.

    A_tilde (N, N): Adj. matrix with self loops
    Returns:
      A_norm : (N, N)
    """

    eps = 1e-5
    d = A_tilde.sum(dim=1) + eps
    D_inv = torch.diag(torch.pow(d, -0.5))
    return (D_inv @ A_tilde @ D_inv).to(torch.float32)

In [ ]:
# To advance the model, use the methods in https://arxiv.org/pdf/2311.02921

class GraphAttentionNetwork(nn.Module):
    """
    HTML‑graph model

        X  ─╮
            │  GAT( 96 → 64 )
            │  ReLU
            │  GAT( 64 → 32 )
            │  ReLU
            └─ Edge‑feature constructor
                      [h_i ‖ h_j ‖ φ(e_ij)] ─► MLP(69 → 1)

    Parameters
    ----------
    in_dim          : node‑feature size   (= 96)
    edge_in_dim     : raw edge‑feature size (= 197)
    edge_emb_dim    : Edge-feature MLP output dims
    """
    def __init__(self,
                 in_dim: int        = 96,
                 edge_in_dim: int   = 197,
                 edge_emb_dim: int  = 8,
                 hidden1: int       = 64,
                 hidden2: int       = 32,
                 heads:  int        = 1):
        super().__init__()

        # ── Node-level encoder (edge-aware) ────────────────────────────
        self.gat1 = GATv2Conv(in_dim,
                              hidden1,
                              heads=heads,
                              concat=True,
                              edge_dim=edge_emb_dim,
                              fill_value="zeros")

        self.gat2 = GATv2Conv(hidden1 * heads,
                              hidden2,
                              heads=1,
                              concat=True,
                              edge_dim=edge_emb_dim,
                              fill_value="zeros")

        # ── Edge feature projector ────────────── (It is not an explicit linear layer as it works on a sparse matrix)
        self.W_edge = nn.Parameter(torch.empty(edge_in_dim, edge_emb_dim))
        nn.init.xavier_uniform_(self.W_edge)

        # ── Edge-level MLP decoder (unchanged) ────────────────────────
        self.edge_mlp = nn.Sequential(
            nn.Linear(hidden2 * 2 + edge_emb_dim, hidden2),
            nn.ReLU(),
            nn.Linear(hidden2, 1)
        )

    # ---------------------------------------------------------------------

    def forward(
        self,
        x_dense: torch.Tensor,        # (N_nodes, 96)          sparse
        A_edge_index: torch.Tensor,   # (2, nnz_A)             COO  (from A)
        A_edge_attr: torch.Tensor,    # (nnz_A, 197)           dense / sparse.mm
        E_edge_index: torch.Tensor,   # (2, N_E)               candidates
        E_edge_attr: torch.Tensor     # (N_E, 197)             sparse features
    ):
        # 1) node features
        #x = x_sparse.to_dense()
        A_edge_emb = torch.sparse.mm(A_edge_attr, self.W_edge)     # (nnz_A , 8)

        # 2) edge-aware GATv2 layers
        h = F.relu(self.gat1(x_dense, A_edge_index, A_edge_emb))
        h = F.relu(self.gat2(h, A_edge_index, A_edge_emb))   # (N_nodes , 32)

        # 3) candidate-edge projection  φ(E) = E @ W_edge
        E_edge_emb = torch.sparse.mm(E_edge_attr, self.W_edge)     # (N_E , 8)

        # 4) gather node embeddings and classify
        src, dst = E_edge_index
        z = torch.cat([h[src], h[dst], E_edge_emb], dim=1)      # (N_E , 72)
        return self.edge_mlp(z).squeeze(-1)                   # (N_E ,) returns the logits

In [ ]:
import copy, torch
import torch.nn.functional as F
from torch import optim
from torch.optim import lr_scheduler
from torch.nn.functional import binary_cross_entropy_with_logits as BCEwLogits

CLIP_NORM = 1.0           # gradient clipping


# ---------- one epoch --------------------------------------------------------
def train_epoch(model, loader, optimizer,
                criterion=BCEwLogits, device="cpu"):

    model.train()
    running_loss, running_edges = 0.0, 0

    for Xs, Aeis, Aefs, Leis, Lefs, ys in loader:
        for X, Aei, Aef, Lei, Lef, y in zip(Xs, Aeis, Aefs, Leis, Lefs, ys):
            X, Aei, Aef = X.to(device), Aei.to(device), Aef.to(device)
            Lei, Lef, y = Lei.to(device), Lef.to(device), y.to(device)

            optimizer.zero_grad()

            logits = model(X, Aei, Aef, Lei, Lef)          # (N_label,)
            loss   = criterion(logits, y.float())

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP_NORM)
            optimizer.step()

            running_loss  += loss.item() * y.numel()
            running_edges += y.numel()

    return running_loss / running_edges


# ---------- evaluation -------------------------------------------------------
@torch.no_grad()
def eval_edge_model(model, loader, device="cpu", thr=0.5):
    model.eval()
    TP = FP = FN = 0

    for Xs, Aeis, Aefs, Leis, Lefs, ys in loader:
        for X, Aei, Aef, Lei, Lef, y in zip(Xs, Aeis, Aefs, Leis, Lefs, ys):
            X, Aei, Aef = X.to(device), Aei.to(device), Aef.to(device)
            Lei, Lef, y = Lei.to(device), Lef.to(device), y.to(device)

            logits = model(X, Aei, Aef, Lei, Lef)
            probs  = torch.sigmoid(logits)

            pred = (probs >= thr).long()
            TP  += ((pred == 1) & (y == 1)).sum().item()
            FP  += ((pred == 1) & (y == 0)).sum().item()
            FN  += ((pred == 0) & (y == 1)).sum().item()

    prec = TP / (TP + FP + 1e-9)
    rec  = TP / (TP + FN + 1e-9)
    f1   = 2 * prec * rec / (prec + rec + 1e-9)
    return prec, rec, f1

In [ ]:
def train_model(model,
                train_loader,
                val_loader,
                num_epochs     = 100,
                lr             = 1e-3,
                validate_every = 10,
                patience       = 10,
                device         = "cpu"):

    model.to(device)
    opt   = optim.AdamW(model.parameters(), lr=lr)
    sched = lr_scheduler.ReduceLROnPlateau(opt, mode="max",
                                          patience=patience, factor=0.5)

    best_f1, best_state, hist = 0.0, None, []

    for epoch in range(1, num_epochs + 1):
        loss = train_epoch(model, train_loader, opt, device=device)
        hist.append(loss)

        if epoch % validate_every == 0 or epoch == num_epochs:
            p, r, f1 = eval_edge_model(model, val_loader, device=device)
            sched.step(f1)

            lr_now = opt.param_groups[0]["lr"]
            print(f"Epoch {epoch:03d}/{num_epochs} "
                  f"loss={loss:.4f}  P={p:.3f} R={r:.3f} F1={f1:.3f}  lr={lr_now:.2e}")

            if f1 > best_f1:
                best_f1, best_state = f1, copy.deepcopy(model.state_dict())

            if lr_now < 1e-5:
                print("Stop: LR < 1e-5")
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    return hist, best_state


In [ ]:
train_dataset = TarGraphDataset(datafile)
train_loader = make_loader(train_dataset, shuffle=False)

# dataset   = TarGraphDataset("/vol/bitbucket/mjh24/IAEA-thesis/data/swde_HTMLgraphs.tar")   # one big archive
# loader    = make_loader(dataset, batch_size=1, shuffle=False)

val_dataset = TarGraphDataset(batchFolders[split:])
val_loader = make_loader(val_dataset, shuffle=False)

model = GraphAttentionNetwork(in_dim = 96, edge_in_dim = 197, edge_emb_dim = 8, hidden1 = 64, hidden2 = 32, heads = 1)

train_model(model,
            train_loader,
            val_loader,
            num_epochs     = 100,
            lr             = 1e-3,
            validate_every = 10,
            patience       = 10,
            device         = "cuda")

IndexError: The shape of the mask [2] at index 0 does not match the shape of the indexed tensor [5448, 8] at index 0